# Universidad Nacional de Chimborazo

**Facultad:** Facultad de Ingeniería  
**Carrera:** Ciencia de Datos e IA  
**Materia:** Modelamiento  
**Docente:** Estalin Mejia H.  
**Semestre:** Tercero

---

## Trabajo Autónomo 2

### Análisis de Regresión Lineal Múltiple: Crímenes de Chicago

**Autores:**  
- Alisson Atupaña
- Mario Camacho
- Lenin Lopez

**Fecha:** 24 de octubre de 2025

**Riobamba - Ecuador**

## 1. Introducción

### Objetivo del Análisis

Realizar un modelo de **regresión lineal múltiple** para predecir la **tasa de arrestos** por distrito en Chicago, utilizando variables como tipo de crimen, ubicación, y factores temporales.

### ¿Qué es Regresión Lineal Múltiple?

Es una técnica que modela la relación entre:
- **Una variable dependiente** (Y): numérica continua
- **Múltiples variables independientes** (X₁, X₂, ..., Xₙ): predictores

**Ecuación:**

$$Y = \beta_0 + \beta_1X_1 + \beta_2X_2 + ... + \beta_nX_n + \epsilon$$

### Dataset: Chicago Crimes (2001-Presente)

- **Fuente:** [Chicago Data Portal](https://data.cityofchicago.org/)
- **Tamaño:** 8+ millones de registros
- **Muestra:** 1,000,000 registros aleatorios

### Variable Dependiente (Y)

**Tasa de Arrestos por Distrito:** Porcentaje de crímenes que resultaron en arresto, agregado por distrito.

### Variables Independientes (X)

1. **Promedio de crímenes domésticos** por distrito
2. **Distribución de tipos de crimen** (proporción de cada tipo)
3. **Promedio de crímenes por hora** del día
4. **Proporción de ubicaciones** más comunes
5. **Índice de densidad criminal** por distrito

## 2. Carga de Librerías

Instalamos y cargamos las librerías necesarias para el análisis:

In [ ]:
# Instalación de paquetes (ejecutar solo la primera vez)
# install.packages(c("readr", "dplyr", "ggplot2", "lubridate", "car", "corrplot"))

# Cargar librerías
library(readr)      # Leer archivos CSV
library(dplyr)      # Manipulación de datos
library(ggplot2)    # Visualizaciones
library(lubridate)  # Manejo de fechas
library(car)        # Diagnósticos de regresión (VIF)
library(corrplot)   # Matriz de correlación

cat("✓ Librerías cargadas correctamente\n")

## 3. Carga de Datos

### Estrategia de Carga Optimizada

Usamos archivo `.RData` para cargar rápidamente la muestra pre-procesada:

In [ ]:
# Archivos
archivo_csv <- "Crimes.csv"
archivo_rdata <- "Chicago_Crimes_Sample_1M.RData"

# Cargar datos
if (file.exists(archivo_rdata)) {
  cat("Cargando muestra pre-procesada...\n")
  load(archivo_rdata)
  cat("✓ Datos cargados:", formatC(nrow(datos_muestra), big.mark=","), "registros\n")
} else {
  cat("Leyendo CSV completo y creando muestra...\n")
  datos_completos <- read_csv(archivo_csv, show_col_types = FALSE)
  
  set.seed(42)
  datos_muestra <- datos_completos %>% sample_n(min(1000000, nrow(datos_completos)))
  
  muestra_info <- list(
    fecha_creacion = Sys.time(),
    registros_originales = nrow(datos_completos),
    registros_muestra = nrow(datos_muestra),
    semilla = 42
  )
  
  save(datos_muestra, muestra_info, file = archivo_rdata)
  rm(datos_completos)
  gc()
  
  cat("✓ Muestra guardada en:", archivo_rdata, "\n")
}

# Exploración inicial
cat("\nDimensiones:", nrow(datos_muestra), "×", ncol(datos_muestra), "\n")
cat("Columnas:", paste(names(datos_muestra), collapse=", "), "\n")

## 4. Preparación de Datos

### 4.1 Limpieza y Transformación

In [ ]:
# Función auxiliar para encontrar columnas
encontrar_col <- function(df, nombres) {
  for(n in nombres) if(n %in% names(df)) return(n)
  return(NULL)
}

# Seleccionar y renombrar columnas necesarias
datos <- datos_muestra %>%
  select(
    Arrest = !!sym(encontrar_col(datos_muestra, c("Arrest", "arrest"))),
    Primary_Type = !!sym(encontrar_col(datos_muestra, c("Primary Type", "Primary.Type"))),
    Location_Description = !!sym(encontrar_col(datos_muestra, c("Location Description", "Location.Description"))),
    Domestic = !!sym(encontrar_col(datos_muestra, c("Domestic", "domestic"))),
    District = !!sym(encontrar_col(datos_muestra, c("District", "district"))),
    Ward = !!sym(encontrar_col(datos_muestra, c("Ward", "ward"))),
    Community_Area = !!sym(encontrar_col(datos_muestra, c("Community Area", "Community.Area"))),
    Date = !!sym(encontrar_col(datos_muestra, c("Date", "date")))
  ) %>%
  mutate(
    # Extraer hora del día
    Date_Time = mdy_hms(Date),
    Hour = hour(Date_Time),
    
    # Convertir tipos
    Arrest = as.logical(Arrest),
    Domestic = as.logical(Domestic),
    District = as.character(District),
    Arrest_Numeric = as.numeric(Arrest)  # 1 = arresto, 0 = no arresto
  ) %>%
  filter(!is.na(District), !is.na(Arrest), District != "") %>%
  drop_na()

cat("✓ Datos limpiados:", formatC(nrow(datos), big.mark=","), "registros\n")

### 4.2 Crear Dataset Agregado por Distrito

**¿Por qué agregamos por distrito?**

Para regresión lineal múltiple necesitamos una **variable dependiente numérica continua**. Calculamos la **tasa de arrestos** (porcentaje) por distrito.

In [ ]:
# Agregación por distrito
datos_distrito <- datos %>%
  group_by(District) %>%
  summarise(
    # VARIABLE DEPENDIENTE: Tasa de arrestos (%)
    Tasa_Arrestos = mean(Arrest_Numeric) * 100,
    
    # VARIABLES INDEPENDIENTES:
    # 1. Proporción de crímenes domésticos
    Prop_Domesticos = mean(as.numeric(Domestic), na.rm=TRUE) * 100,
    
    # 2. Total de crímenes (densidad)
    Total_Crimenes = n(),
    
    # 3. Hora promedio de ocurrencia
    Hora_Promedio = mean(Hour, na.rm=TRUE),
    
    # 4. Diversidad de tipos de crimen (índice)
    Diversidad_Crimenes = n_distinct(Primary_Type),
    
    # 5. Diversidad de ubicaciones
    Diversidad_Ubicaciones = n_distinct(Location_Description)
  ) %>%
  filter(Total_Crimenes >= 100) %>%  # Solo distritos con suficientes datos
  mutate(
    # Normalizamos algunas variables
    Log_Crimenes = log(Total_Crimenes + 1),
    Densidad_Normalizada = scale(Total_Crimenes)[,1]
  )

cat("\n✓ Dataset agregado creado\n")
cat("  Distritos:", nrow(datos_distrito), "\n")
cat("  Variables:", ncol(datos_distrito), "\n\n")

# Mostrar primeras filas
print(head(datos_distrito, 10))

## 5. Análisis Exploratorio

### 5.1 Estadísticas Descriptivas

In [ ]:
cat("=== ESTADÍSTICAS DESCRIPTIVAS ===\n\n")
summary(datos_distrito %>% select(-District))

### 5.2 Visualización de Relaciones

In [ ]:
# Gráfico 1: Distribución de tasa de arrestos
ggplot(datos_distrito, aes(x=Tasa_Arrestos)) +
  geom_histogram(bins=15, fill="steelblue", color="white") +
  labs(title="Distribución de Tasa de Arrestos por Distrito",
       x="Tasa de Arrestos (%)", y="Frecuencia") +
  theme_minimal()

# Gráfico 2: Relación entre variables
ggplot(datos_distrito, aes(x=Prop_Domesticos, y=Tasa_Arrestos)) +
  geom_point(size=3, alpha=0.6, color="darkblue") +
  geom_smooth(method="lm", se=TRUE, color="red") +
  labs(title="Tasa de Arrestos vs Proporción de Crímenes Domésticos",
       x="Proporción de Crímenes Domésticos (%)",
       y="Tasa de Arrestos (%)") +
  theme_minimal()

### 5.3 Matriz de Correlación

In [ ]:
# Seleccionar variables numéricas
vars_numericas <- datos_distrito %>%
  select(Tasa_Arrestos, Prop_Domesticos, Log_Crimenes, 
         Hora_Promedio, Diversidad_Crimenes, Diversidad_Ubicaciones)

# Calcular matriz de correlación
matriz_cor <- cor(vars_numericas, use="complete.obs")

# Visualizar
corrplot(matriz_cor, method="color", type="upper", 
         addCoef.col="black", number.cex=0.7,
         tl.col="black", tl.srt=45,
         title="Matriz de Correlación",
         mar=c(0,0,2,0))

cat("\n📊 Correlaciones con Tasa_Arrestos:\n")
print(sort(matriz_cor[,"Tasa_Arrestos"], decreasing=TRUE))

## 6. Modelo de Regresión Lineal Múltiple

### 6.1 Construcción del Modelo

In [ ]:
# Modelo completo
modelo <- lm(Tasa_Arrestos ~ Prop_Domesticos + Log_Crimenes + 
             Hora_Promedio + Diversidad_Crimenes + Diversidad_Ubicaciones,
             data = datos_distrito)

# Resumen del modelo
cat("\n=== RESUMEN DEL MODELO DE REGRESIÓN LINEAL MÚLTIPLE ===\n\n")
summary(modelo)

### 6.2 Interpretación de Métricas

#### R² (R-squared)
- Indica qué porcentaje de la variabilidad en Y es explicada por el modelo
- Valor entre 0 y 1 (más cercano a 1 = mejor)

#### R² Ajustado (Adjusted R-squared)
- Penaliza la inclusión de variables que no mejoran el modelo
- Más confiable que R² cuando hay múltiples predictores

#### p-value
- < 0.05: La variable es estadísticamente significativa
- > 0.05: La variable no aporta significativamente al modelo

### 6.3 Diagnósticos del Modelo

In [ ]:
# Gráficos de diagnóstico
par(mfrow=c(2,2))
plot(modelo)
par(mfrow=c(1,1))

# VIF (Variance Inflation Factor) - Detectar multicolinealidad
cat("\n=== FACTOR DE INFLACIÓN DE VARIANZA (VIF) ===\n")
cat("VIF < 5: No hay multicolinealidad\n")
cat("VIF > 10: Multicolinealidad severa\n\n")
print(vif(modelo))

## 7. Resultados y Conclusiones

### 7.1 Ecuación del Modelo

In [ ]:
coefs <- coef(modelo)
cat("\n=== ECUACIÓN DEL MODELO ===\n\n")
cat(sprintf("Tasa_Arrestos = %.2f + %.2f(Prop_Domesticos) + %.2f(Log_Crimenes) + \n",
            coefs[1], coefs[2], coefs[3]))
cat(sprintf("                %.2f(Hora_Promedio) + %.2f(Diversidad_Crimenes) + \n",
            coefs[4], coefs[5]))
cat(sprintf("                %.2f(Diversidad_Ubicaciones)\n", coefs[6]))

### 7.2 Predicciones vs Valores Reales

In [ ]:
# Añadir predicciones al dataset
datos_distrito$Prediccion <- predict(modelo)
datos_distrito$Residuos <- residuals(modelo)

# Gráfico de predicciones vs reales
ggplot(datos_distrito, aes(x=Tasa_Arrestos, y=Prediccion)) +
  geom_point(size=3, alpha=0.6, color="darkblue") +
  geom_abline(intercept=0, slope=1, color="red", linetype="dashed", size=1) +
  labs(title="Valores Predichos vs Valores Reales",
       x="Tasa de Arrestos Real (%)",
       y="Tasa de Arrestos Predicha (%)") +
  theme_minimal()

### 7.3 Métricas Finales

In [ ]:
# Extraer métricas
r2 <- summary(modelo)$r.squared
r2_adj <- summary(modelo)$adj.r.squared
rmse <- sqrt(mean(residuals(modelo)^2))
mae <- mean(abs(residuals(modelo)))

cat("\n=== MÉTRICAS FINALES DEL MODELO ===\n\n")
cat(sprintf("R² (R-cuadrado):           %.4f (%.2f%%)\n", r2, r2*100))
cat(sprintf("R² Ajustado:               %.4f (%.2f%%)\n", r2_adj, r2_adj*100))
cat(sprintf("RMSE (Error cuadrático):   %.2f\n", rmse))
cat(sprintf("MAE (Error absoluto):      %.2f\n", mae))

cat("\n📊 Interpretación:\n")
if(r2_adj > 0.7) {
  cat("✓ Excelente ajuste del modelo (R² ajustado > 0.70)\n")
} else if(r2_adj > 0.5) {
  cat("✓ Buen ajuste del modelo (R² ajustado > 0.50)\n")
} else {
  cat("⚠️ Ajuste moderado del modelo (R² ajustado < 0.50)\n")
}

## 8. Conclusiones

### Hallazgos Principales

1. **Variables Significativas:** Identificamos qué factores influyen más en la tasa de arrestos
2. **Calidad del Modelo:** R² y R² ajustado indican qué tan bien el modelo explica los datos
3. **Aplicabilidad:** El modelo puede ayudar a predecir tasas de arresto en diferentes distritos

### Limitaciones

- Agregación por distrito reduce el tamaño muestral
- Variables omitidas pueden influir en los resultados
- Relaciones pueden no ser perfectamente lineales

### Recomendaciones

- Incluir más variables socioeconómicas
- Probar transformaciones de variables
- Considerar interacciones entre predictores